In [3]:
import praw
import pandas as pd
import datetime as dt

In [4]:
reddit = praw.Reddit(client_id='73IApaJarJRRVA',client_secret='QIET4TRTvxlLWDNFfobmDD7oAfM',user_agent='PD sentiment',username='hll4ce',password='Snsh11nE')


In [6]:
subreddit = reddit.subreddit('Parkinsons')

top_subreddit = subreddit.top(limit=1000)

In [7]:
for submission in subreddit.top(limit=1):
    print(submission.title, submission.id)

When life throws you Parkinson's and you can't hold a pen steady enough for lines.....why not just use dots!!! 18x24 inch pen and ink pointillism. a2c528


In [8]:
topics_dict = { "title":[],"score":[], "id":[], "url":[],"comms_num": [], "created": [], "body":[]}
 
    
    
    
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

topics_data = pd.DataFrame(topics_dict)


In [9]:
topics_data

,title,score,id,url,comms_num,created,body
0,When life throws you Parkinson's and you can't...,70,a2c528,https://i.redd.it/oo39m4zm9u121.jpg,13,1.543774e+09,
1,My mom lost her battle with Parkinson’s today,59,9zjuqj,https://www.reddit.com/r/Parkinsons/comments/9...,31,1.542965e+09,"After a long battle with Parkinson’s, my mom p..."
2,Michael J. Fox raised 800 million!,55,bevavd,https://www.reddit.com/r/Parkinsons/comments/b...,2,1.555678e+09,Him and his wife may be the the best couples f...
3,Irish drug that cures Parkinson's in mice gets...,57,b6g66h,https://www.irishcentral.com/news/dublin-parki...,5,1.553790e+09,
4,She's dead,47,9iz9fr,https://www.reddit.com/r/Parkinsons/comments/9...,8,1.537964e+09,My best friend died two weeks ago. She suffere...
5,About right lol.,45,ap05ap,https://i.redd.it/sz4weimzsnf21.jpg,19,1.549798e+09,
6,My dad passed away today after a 5 years strug...,44,earge6,https://www.reddit.com/r/Parkinsons/comments/e...,8,1.576397e+09,I know he’s not in pain anymore and that give...
7,Lost my dad this week to PD. Here is info on h...,43,dkk8qh,https://www.reddit.com/r/Parkinsons/comments/d...,14,1.571610e+09,My dad was diagnosed at 68 and died at 73. My...
8,The battle is over,41,bdv39r,https://www.reddit.com/r/Parkinsons/comments/b...,15,1.555456e+09,My dad lived across the hall from me in my apa...
9,I'm thriving with Parkinson's...it's been 24 y...,41,arnt4w,https://www.reddit.com/r/Parkinsons/comments/a...,11,1.550459e+09,"Through a combination of exercise, meditation,..."
